In [14]:
import pandas as pd
import os
import re

def extract_parts_with_labels_from_documents():
    data = []
    doc_counter = 0
    
    # Parcours de tous les fichiers dans le dossier de documents
    for filename in os.listdir():
    
        if filename.endswith(".txt"):  # Assurez-vous que le fichier est un fichier texte
            with open(os.path.join(filename), 'r', encoding='utf-8') as file:
                doc_content = file.read()
                
                # Extraire les parties avec leurs labels
                pattern = r'<para>\s*(\d+)\s+(.*?)<\\para>'
                matches = re.findall(pattern, doc_content, re.DOTALL)
                
                # Ajouter les données extraites à la liste
                for match in matches:
                    label = match[0]
                    part = match[1].strip()
                    data.append((part, int(label), doc_counter))
                    
            doc_counter += 1  # Incrémenter le numéro du document
    
    # Créer un DataFrame Pandas à partir des données
    df = pd.DataFrame(data, columns=['Contenu du paragraphe', 'Label', 'Numéro du document'])
    
    return df

# Exemple d'utilisation avec un dossier contenant des documents

# Extraire les parties et les labels de tous les documents
df = extract_parts_with_labels_from_documents()




In [15]:
df.to_csv("dataset.csv")

In [2]:
reviews = df['Contenu du paragraphe'].values.tolist()
sentiments = df['Label'].values.tolist()

In [57]:
len(df)

232

In [58]:
len(df[df["Label"] == 1])

49

In [61]:
len(df[df["Label"] == 0])

183

In [62]:
49/183

0.2677595628415301

In [3]:
import pandas as pd
import torch
from tqdm import tqdm, trange
import numpy as np


# Importing specific libraries for data prerpcessing, model archtecture choice, training and evaluation
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW

In [4]:
epochs = 5
MAX_LEN = 128
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Spliting Training and validation data

In [5]:
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

In [7]:
df.columns

Index(['Contenu du paragraphe', 'Label', 'Numéro du document'], dtype='object')

In [8]:
text = df['Contenu du paragraphe'].to_list()
labels = df['Label'].to_list()

#user tokenizer to convert sentences into tokenizer
input_ids  = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN) for sent in text]

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]  
    attention_masks.append(seq_mask)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [9]:
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks,
                                                            random_state=42, test_size=0.1)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# CamemBERT model

In [10]:
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=2)
model.to(device)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CamembertForSequenceClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=Tru

In [11]:
device

device(type='cpu')

# Defining the parameters and metrics to optimize

In [12]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

/Users/seydinaousmaneniang/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Training and evaluating our model

In [13]:
# Store our loss and accuracy for plotting if we want to visualize training evolution per epochs after the training process
train_loss_set = []

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):  
    # Tracking variables for training
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    # Train the model
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # Get loss value
        loss = outputs[0]
        # Add it to train loss list
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
    
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    


    # Tracking variables for validation
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Validation of the model
    model.eval()
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs =  model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs[:2]
    
        # Move logits and labels to CPU if GPU is used
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|                                              | 0/5 [00:00<?, ?it/s]

Train loss: 0.6157316186211326


Epoch:  20%|███████▌                              | 1/5 [01:29<05:58, 89.63s/it]

Validation Accuracy: 0.6458333333333333
Train loss: 0.5292431901801716


Epoch:  40%|███████████████▏                      | 2/5 [02:56<04:24, 88.04s/it]

...

Validation Accuracy: 0.6458333333333333
Train loss: 0.43375999277288263


Epoch:  80%|██████████████████████████████▍       | 4/5 [05:54<01:28, 88.87s/it]

Validation Accuracy: 0.7395833333333333
Train loss: 0.2954600710760463


Epoch: 100%|██████████████████████████████████████| 5/5 [07:26<00:00, 89.28s/it]

Validation Accuracy: 0.96875


# Test the model on an unseen texts

In [53]:
comments = ["présence d'un stimulateur cardiaque implanté dans le corps", 
           " peacemaker  implanté dans le corps",
            "radiographie cardiaque"
           ]

In [54]:
tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True,max_length=MAX_LEN) for comment in comments]
# Pad the resulted encoded comments
tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks 
attention_masks = []
for seq in tokenized_comments_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(tokenized_comments_ids)
prediction_masks = torch.tensor(attention_masks)

In [55]:
flat_pred = []
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs =  model(prediction_inputs.to(device),token_type_ids=None, attention_mask=prediction_masks.to(device))
    logits = outputs[0]
    logits = logits.detach().cpu().numpy() 
    flat_pred.extend(np.argmax(logits, axis=1).flatten())

In [56]:
for i in range(len(flat_pred)):
    print('Comment: ', comments[i])
    print('Label', flat_pred[i])

Comment:  présence d'un stimulateur cardiaque implanté dans le corps
Label 1
Comment:   peacemaker  implanté dans le corps
Label 1
Comment:  radiographie cardiaque
Label 0


In [63]:
def model(comments):
    tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True,max_length=MAX_LEN) for comment in comments]
    # Pad the resulted encoded comments
    tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # Create attention masks 
    attention_masks = []
    for seq in tokenized_comments_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    prediction_inputs = torch.tensor(tokenized_comments_ids)
    prediction_masks = torch.tensor(attention_masks)
    flat_pred = []
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs =  model(prediction_inputs.to(device),token_type_ids=None, attention_mask=prediction_masks.to(device))
        logits = outputs[0]
        logits = logits.detach().cpu().numpy() 
        flat_pred.extend(np.argmax(logits, axis=1).flatten())
    return flat_pred

In [64]:
def extraire_paragraphes(texte):
    paragraphes = re.split(r'\n\s*\n', texte)  # Sépare le texte en paragraphes
    paragraphes_pertinents = []
    for paragraphe in paragraphes:
        if re.search(r'\b(cardiaque|stimulateur\s+cardiaque)\b', paragraphe, re.IGNORECASE):
            paragraphes_pertinents.append(paragraphe)
    return paragraphes_pertinents

In [ ]:
def medical_report_analysis(text):
    paragraphs = extraire_paragraphes(text)
    pred = model(paragraphs)
    return max(pred)